https://github.com/veribilimiokulu/udemy-apache-spark/blob/master/kurs_esnasi/pyspark/MachineLearning/Regression/MultipleLineerRegresyon.ipynb

###Yalnızca korelasyon katsayısı pozitif veya negatif 0,50 veya ötesindeyse regresyon analizi yapılır.###

In [2]:
!pip install findspark

import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark import SparkContext


sc = SparkContext('local')
spark = SparkSession(sc)
sc

<SparkContext master=local appName=pyspark-shell>

In [3]:
df =spark.read.csv("advertising.csv", header=True , inferSchema = True, sep = "," )

In [4]:
df.toPandas().head ()

,ID,TV,Radio,Newspaper,Sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


## Sütun İsimlerini Değiştirme

In [5]:
yeni_nitelikler = ["id", "TV","Radio","Newspaper","label"]

In [6]:
df2 = df.selectExpr("ID as id","TV","Radio","Newspaper","Sales as label")
df2.toPandas().head()

,id,TV,Radio,Newspaper,label
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


## Numerik Niteliklerle Hedef Nitelikleri Ayrı Ayrı tutalım

In [26]:
# daha sonra kullanmak üzere ayırıyoruz
numerik_nitelikler = ["TV","Radio"]
# eskiden newspaper adında bir sütun daha vardı bu listede ancak p değeri yüksek çıkınca kaldırdık
label = ["label"]


In [8]:
# Veri keşfi yapıyoruz
df2.describe().toPandas().head()
# countların hepsi 200 olduğu için null değer olmadığı sonucuna vardık

,summary,id,TV,Radio,Newspaper,label
0,count,200,200,200,200,200
1,mean,100.5,147.0425,23.264000000000024,30.553999999999995,14.022500000000003
2,stddev,57.879184513951124,85.85423631490805,14.846809176168728,21.77862083852283,5.217456565710477
3,min,1,0.7,0.0,0.3,1.6
4,max,200,296.4,49.6,114.0,27.0


## Veri hazırlığı
Elimizide hiç kategorik değişken yok, kategorik değişken olmadığı için hazırlık yapmak zorunda değiliz
Yani StringIndexer ve OnehotEncoder kullanma gereğimiz yok.Doğrudan vector assambler'e geçiyoruz. Ve nümerik sütunları
aynı vektör içerisinde topluyoruz 

In [27]:
# import edip nesnesini oluşturuyoruz
from pyspark.ml.feature import VectorAssembler
vector_assembler = VectorAssembler().setInputCols(numerik_nitelikler).setOutputCol("features")

 ##  Regresyon Modeli Oluşturuyoruz 

In [28]:
## import edip nesnesini oluşturuyoruz
from pyspark.ml.regression import LinearRegression
lr_obj = LinearRegression().setFeaturesCol("features").setLabelCol("label")

## Pipeline Model Oluşturuyoruz

In [29]:
## import edip nesnesini oluşturuyoruz
from pyspark.ml import Pipeline
pipeline_obj = Pipeline().setStages([vector_assembler, lr_obj])

## veri setini test ve train olarak  ayırma

In [30]:
train_df, test_df = df2.randomSplit([0.8, 0.2], seed=142)

## Pipeline Modelini  eğitme

In [31]:
pipeline_model = pipeline_obj.fit(train_df)

## Model testi

In [32]:
result_df = pipeline_model.transform(test_df)
result_df.toPandas().head()

,id,TV,Radio,Newspaper,label,features,prediction
0,4,151.5,41.3,58.5,18.5,"[151.5, 41.3]",17.650134
1,9,8.6,2.1,1.0,4.8,"[8.6, 2.1]",3.716108
2,11,66.1,5.8,24.2,8.6,"[66.1, 5.8]",7.043478
3,15,204.1,32.9,46.0,19.0,"[204.1, 32.9]",18.469043
4,25,62.3,12.6,18.3,9.7,"[62.3, 12.6]",8.153565


## Pipeline model içinden lineer modeli almak, Stages sorgusu soncunda

In [33]:
# stages ile lineer modelin indis numarasını sorguluyoruz
pipeline_model.stages

[VectorAssembler_ef8642e9532c,
 LinearRegressionModel: uid=LinearRegression_3506be163033, numFeatures=2]

In [34]:
# 0. indiste vectorassmbler 1. indisti ise lineerregression modeli varmış.
# şimdi ÇAĞIRMASI KOLAY OLSUN DİYE BİR NESNEYE ATAyalım
lr_model = pipeline_model.stages[1]

In [35]:
#COEFFICIENT DEĞERİNİ SORGULAYALIM 
lr_model.coefficients

DenseVector([0.0457, 0.1888])

In [36]:
#sabiti görelim 
lr_model.intercept

2.926454603092079

In [37]:
# Toplam değişkenliği 1 olarak kabul ettiğimizde , bizim modelimiz bu değişkenliğin %90'ini açıklayabiliyormuş.
lr_model.summary.r2

0.9063946363969204

In [38]:
# Bu model anlamlı mı değil mi? p değerlerine bakalım :
lr_model.summary.pValues
#9. satırda vectorassembler nesnesinde kullanmak üzere önceden 7. satırda numerik değerler  adında bir listeyle 
#bu hesaplanacak p değerlerinin listesini tanımlamıştık

[0.0, 0.0, 0.0]

1. p değeri 0 olmuş
2. p değeri 0 olmuş

bu p değerleri 0.05'in altında 

3. p değeri 0.05'in çok üstünde
4. p değeri 0.05'in çok altında çünkü en sonunda (e-16) yazıyor

demek ki en başta 7. satıra gidip nümerik değerler listesinde baştan 3. gelen "Newspaper sütunun kaldırıp
yeniden çalıştıracağız. böylece p değerlerinin hepsi 0.05'ten küçük olanları ML modelinde kullanmış olacağız

In [39]:
#  değerine bakalım 
lr_model.summary.tValues

[31.28678417062139, 22.454684697674782, 9.309569401834192]

In [40]:
# .rootMeanSquaredError , bu rakamın label (sonuç) değişkenin standart sapmasından küçük olması gerekiyor
#8. satırda standart sapma gözüküyor.
lr_model.summary.rootMeanSquaredError

1.5678514278266518

## Model Seçimi
Geriye doğru eleme yönteminde Newspaper p değeri 0.05 değerinden büyük olduğu (0.5717) için modelden çıkarılır. Bunun için nümerik nitelikler listesinden Newspaper'ı çıkarmamız yeterli. Daha sonra o paragraftan itibaren tekrar çalıştıralım.

y = 2.935593134859488 + (0.0442 * TV) + (0.1978 * Radio)

## Prediction

In [41]:
# 100 bin TV için 10 bin Radio için
# diyelim ki 100bin tv reklamı için 10 bin de radyo reklamı için bütçe ayırırsak, tahminen satış rakamlarımız ne olır? 
import pandas as pd
d = {"TV":[100.0], "Radio":[10.0]}
pd_df = pd.DataFrame(data=d)
pd_df.head()

,TV,Radio
0,100.0,10.0


In [42]:
# şimdi bu pandas dataframe'i , spark dataframe'e çevirelim:
predict_df = spark.createDataFrame(pd_df)
predict_df.show()

+-----+-----+
|   TV|Radio|
+-----+-----+
|100.0| 10.0|
+-----+-----+



## LR_MODEL'İ ÇALIŞTIRALIM 

In [43]:
# predict yapalım
#önceden vectorassembler hazırlamıştı, bununla dataframe'i transform edeceğiz sonra
# daha sonra transform edilmiş data frame'i  modelimizin içine verececeğiz 
predict_df_vec = vector_assembler.transform(predict_df)

In [44]:
lr_model.transform(predict_df_vec).show()

+-----+-----+------------+-----------------+
|   TV|Radio|    features|       prediction|
+-----+-----+------------+-----------------+
|100.0| 10.0|[100.0,10.0]|9.386286831310258|
+-----+-----+------------+-----------------+



100 bin tv üzerinden reklam verme için , 10 bin de radyo üzerinden reklam verme bütçesi ayırırsam , tahmini satış adedim 9.38 bin olacakmış